In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(37) # 使得每次运行得到的随机数都一样

In [2]:
# 1 准备数据集
# 从文件中加载数据集
feature_data_path='E:\PyProjects\DataSet\BuildingInOut/CalIt2.data'
feature_set=pd.read_csv(feature_data_path,header=None)
print(feature_set.info())
# print(feature_set.head()) 
# print(feature_set.tail()) # 检查没有问题

label_data_path='E:\PyProjects\DataSet\BuildingInOut/CalIt2.events'
label_set=pd.read_csv(label_data_path,header=None)
print(label_set.info())
# print(label_set.head())
# print(label_set.tail()) # 读取没有问题，但是第三列有问题，没有将数字和event分开。


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 4 columns):
0    10080 non-null int64
1    10080 non-null object
2    10080 non-null object
3    10080 non-null int64
dtypes: int64(2), object(2)
memory usage: 315.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
0    30 non-null object
1    30 non-null object
2    30 non-null object
3    30 non-null object
dtypes: object(4)
memory usage: 1.0+ KB
None


In [3]:
# 对label_set中第二列去除后面的分号
need_split_col=label_set.iloc[:,3]
label_set.iloc[:,3]=need_split_col.map(lambda x: int(x.split(';')[0].strip()))
label_set[4]=need_split_col.map(lambda x: x.split(';')[1].strip())
# print(label_set.info()) # 此处的第三列10,20等整数估计代表event的代号，此处用不上
# print(label_set.head()) # 处理后正常


In [4]:
# 将feature_set中进楼和出楼的人员统计到一行。
# 目前数据集中出楼人员（代号7）位于偶数行，进楼人员（代号9）位于奇数行。
# 可以使用for in 方式依次取出各行的人员数，但此处我更愿意使用布尔型索引
code_7=feature_set[0]==7
code_7_data=feature_set[code_7].iloc[:,3].reset_index(drop=True)
code_9=feature_set[0]==9
code_9_data=feature_set[code_9].reset_index(drop=True)
# print(code_9_data)# OK
feature_set2=code_9_data
feature_set2[4]=code_7_data
feature_set2.drop([0],axis=1,inplace=True) # 删除第0列
# print(feature_set2) # col3 表示in，col4表示out，打印没有问题
# feature_set2.to_csv('d:/feature_set2.csv') # 保存以便查看是否有误

In [5]:
# 下面是如何将feature_set2和label_set整合到一个DataFrame中来
# 要判断时间，如果feature_set2中的日期和时间都落在了label_set对应的时间内，
# 则表示有event发生，用1表示，如果没有，用0表示。
# 比较日期时间的方法有很多，此处我采用比较简单的方法
feature_set2[5]=0 # 表示是否有event的列都初始化为0

def calc_mins(time_str):
    nums=time_str.split(':')
    return 60*int(nums[0])+int(nums[1]) # 将时间转换为分钟数，此处不用考虑秒

for row_id,date in enumerate(label_set[0]): # 先取出label中的日期
    temp_df=feature_set2[feature_set2[1]==date]
    if temp_df is None:
        continue
    
    start_min=calc_mins(label_set.iloc[row_id,1])
    stop_min=calc_mins(label_set.iloc[row_id,2])
    for row in temp_df[2]: # 在逐一判断时间是否位于label中时间之间
        feature_min=calc_mins(row)
        if feature_min>=start_min and feature_min<=stop_min: 
            feature_row=temp_df[temp_df[2]==row].index.tolist()
            feature_set2.loc[feature_row,5]=1 
        
# feature_set2.to_csv('d:/feature_set2_withLabel.csv') # 保存后打印查看没有问题
        

In [6]:
# 进一步处理，由于日期在以后的日子里不可重复，作为feature并不合适，而可以用星期数来代替，
feature_set2[0]=pd.to_datetime(feature_set2[1])
# print(feature_set2.tail())
feature_set2[0]=feature_set2[0].map(lambda x: x.strftime('%w')) # 将日期转换为星期数
feature_set2=feature_set2.reindex(columns=range(6))
print(feature_set2.tail()) # 查看转换没有问题
feature_set2.to_csv('E:\PyProjects\DataSet\BuildingInOut/Sorted_Set.txt') # 将整理好的数据集保存，下次可以直接读取

      0         1         2  3  4  5
5035  6  11/05/05  21:30:00  0  0  0
5036  6  11/05/05  22:00:00  0  3  0
5037  6  11/05/05  22:30:00  0  0  0
5038  6  11/05/05  23:00:00  0  0  0
5039  6  11/05/05  23:30:00  0  1  0


In [7]:
# 由于第1列只是包含日期，作为特征向量并不合适，故而需要删除
feature_set2.drop([1],axis=1,inplace=True)
# 而第2列明显是字符串类型，里面的内容对机器学习而言如同天书，故需要编码
from sklearn import preprocessing
time_encoder=preprocessing.LabelEncoder()
feature_set2[2]=time_encoder.fit_transform(feature_set2[2])
print(feature_set2.tail())

      0   2  3  4  5
5035  6  43  0  0  0
5036  6  44  0  3  0
5037  6  45  0  0  0
5038  6  46  0  0  0
5039  6  47  0  1  0


In [8]:
dataset_X,dataset_y=feature_set2.iloc[:,:-1].values,feature_set2.iloc[:,-1].values
# 拆分数据集为train set和test set
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y=train_test_split(dataset_X,dataset_y,
                                                  test_size=0.3,random_state=42)
# print(train_X.shape) # (3528, 4)
# print(train_y.shape) # (3528,)
# print(test_X.shape) # (1512, 4)

In [9]:
# 下面是使用SVM构建分类器
from sklearn.svm import SVC
classifier=SVC(kernel='rbf',probability=True,class_weight='balanced')
classifier.fit(train_X,train_y)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
# 用交叉验证来检验模型的准确性，只是在test set上验证准确性
from sklearn.cross_validation import cross_val_score
num_validations=5
accuracy=cross_val_score(classifier,test_X,test_y,
                         scoring='accuracy',cv=num_validations)
print('准确率：{:.2f}%'.format(accuracy.mean()*100))
precision=cross_val_score(classifier,test_X,test_y,
                         scoring='precision_weighted',cv=num_validations)
print('精确度：{:.2f}%'.format(precision.mean()*100))
recall=cross_val_score(classifier,test_X,test_y,
                         scoring='recall_weighted',cv=num_validations)
print('召回率：{:.2f}%'.format(recall.mean()*100))
f1=cross_val_score(classifier,test_X,test_y,
                         scoring='f1_weighted',cv=num_validations)
print('F1  值：{:.2f}%'.format(f1.mean()*100))


e:\programsoft\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


准确率：93.78%
精确度：92.96%
召回率：93.78%
F1  值：92.96%


In [11]:
# 看起来该模型的预测效果很不错
# 那么用它来对新样本数据进行预测，会是什么样了？
new_samples=np.array([[2,'09:30:00',20,12], # 即某个星期二，上午9点-9点半时间段，进大楼20人，出大楼12人
             [2,'11:30:00',26,9],
             [6,'12:30:00',4,22],
             [0,'05:00:00',1,0]])
transformed=time_encoder.transform(new_samples[:,1])
# print(transformed) # 检查OK
new_samples[:,1]=transformed
print(new_samples)

# 使用classifier进行预测
output_class = classifier.predict(new_samples)
print('has events? {}'.format(output_class))

[['2' '19' '20' '12']
 ['2' '23' '26' '9']
 ['6' '25' '4' '22']
 ['0' '10' '1' '0']]
has events? [0 1 0 0]
